<a href="https://colab.research.google.com/github/shabayadletzh/Midterm-Project/blob/main/Midterm_Data_Bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


##Midterm Project
####Data Bootcamp (ECON-UB  232)
####Prof. Jacob Koehler
####Ahmad Alshikh Menou, Adilet Shabay, Arnur Abdrakhman


---



**Abstract:**

*Financial data is scattered across filings, feeds, and formats, which makes it hard to compare companies, connect fundamentals to market performance, and build portfolios that reflect investor preferences. This project delivers a unified, reproducible pipeline that pulls fundamentals from SEC EDGAR and daily prices from a public feed, then transforms them into clear insights and interactive visuals. The system selects the freshest revenue tag from 10-K and 10-Q data, standardizes quarterly and annual revenue, and computes key metrics such as net margin, current ratio, and debt to equity. It extracts concise management notes from the latest 10-K using keyword scoring, displays revenue and price history with interactive sliders, and provides a transparent rules based Buy, Hold, or Sell label. A simple linear regression forecasts the next revenue figure and reports confidence and prediction intervals, which highlights trend and uncertainty. A second module constructs a diversified, long only equity portfolio from a broad universe using a risk aversion input that maps to blended weights across mean variance, inverse volatility, and equal weight sleeves. The tool reports growth of one dollar, volatility, Sharpe ratio, and drawdown. Known limits include no dividends, no fees, potential survivorship bias, and a simplified rating and forecast.*

*The problem we are trying to solve is streamling and easing the process of financial analysis by providing tools for looking financial information of publicly traded companies and attempting to forecast their revenues. Furthermore we provide a portfolio tool which the user can access to visualize the key metrics of their assets.*


## Individual Company Performance Lookup Tool







This first code builds the end-to-end data pipeline by talking directly to the SEC EDGAR APIs and shaping the results into clean series and charts. It maps a ticker to its CIK using https://www.sec.gov/files/company_tickers.json, looks up the company’s latest forms in https://data.sec.gov/submissions/CIK{CIK}.json, and downloads fundamentals from https://data.sec.gov/api/xbrl/companyfacts/CIK{CIK}.json. From those facts it examines several GAAP revenue tags and keeps the one with the most recent period so the figures reflect the newest filing. It then standardizes revenue into two comparable views, quarterly numbers created by differencing year to date 10-Q values within each fiscal year, and annual numbers taken once per fiscal year with a preference for 10-K rows. The cell also pulls the latest values for net income, assets, liabilities, equity, cash, operating cash flow, current assets, current liabilities, and diluted EPS, and computes simple checks like current ratio, debt to equity, net margin, and year over year revenue growth. To connect fundamentals with the market it fetches daily closing prices for the ticker from a public feed and organizes everything into tidy DataFrames ready for interactive plots and summary stats that appear in later cells.

We believe this tool provides a much faster and more streamlined approach to looking up the key financial information of a public company as the user does not need to go through a third-party paid platform or websites full of advertisments to get the desired data.

In [ ]:
import io, os, re, time, json, random
from datetime import datetime, timedelta
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import pandas as pd
import requests
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, DateRangeSlider
from bokeh.layouts import column
from typing import List, Tuple, Optional

In [19]:
output_notebook()  # show bokeh plots inside the notebook

# basic config
PAUSE_S = 0.2  # brief pause to be polite with sec servers
SEC_USER_AGENT = os.getenv("SEC_USER_AGENT", "Ahmad Alshikh Menou aa11184@nyu.edu (academic use)")  # sec requires this
HEADERS = {"User-Agent": SEC_USER_AGENT, "Accept-Encoding": "gzip, deflate"}

# revenue tags to try (we pick the freshest one)
REVENUE_TAGS = ["RevenueFromContractWithCustomerExcludingAssessedTax", "SalesRevenueNet", "Revenues"]

# http helper
def get_json(url: str) -> dict:
    time.sleep(PAUSE_S)
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

# ticker → cik mapping (pull official sec list, with a fallback)
def get_ticker_map() -> pd.DataFrame:
    primary = "https://www.sec.gov/files/company_tickers.json"
    try:
        data = get_json(primary)
        df = pd.DataFrame(list(data.values()))
        df["cik"] = df["cik_str"].astype(str).str.zfill(10)
        df = df.rename(columns={"ticker":"ticker", "title":"company"})
        return df[["ticker","cik","company"]]
    except Exception:
        fallback = "https://www.sec.gov/files/company_tickers_exchange.json"
        data = get_json(fallback)
        df = pd.DataFrame(data)
        df["cik"] = df["cik"].astype(str).str.zfill(10)
        df["ticker"] = df["ticker"].str.upper()
        df = df.rename(columns={"title":"company"})
        return df[["ticker","cik","company"]]

# find cik for a given ticker (case-insensitive)
def cik_for_ticker(ticker: str, tickers_df: pd.DataFrame) -> Optional[str]:
    row = tickers_df[tickers_df["ticker"].str.upper() == ticker.upper()]
    return None if row.empty else row["cik"].iloc[0]

# company facts (xbrl) from edgar
def get_companyfacts(cik: str) -> dict:
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    return get_json(url)

# choose the freshest revenue tag and return a tidy dataframe of rows
def pick_freshest_revenue_tag(facts: dict, tags: List[str]) -> Tuple[Optional[str], Optional[pd.DataFrame]]:
    usg = facts.get("facts", {}).get("us-gaap", {})
    best_tag, best_df = None, None
    best_last = pd.Timestamp("1900-01-01")
    for tag in tags:
        node = usg.get(tag)
        if not node:
            continue
        units = node.get("units", {})
        usd_keys = [k for k in units if "usd" in k.lower()]
        if not usd_keys:
            continue
        rows = []
        for uk in usd_keys:
            rows.extend(units.get(uk, []))
        df = pd.DataFrame(rows)
        if df.empty or "end" not in df or "val" not in df:
            continue
        df["end"] = pd.to_datetime(df["end"], errors="coerce")
        df["val"] = pd.to_numeric(df["val"], errors="coerce")
        df = df.dropna(subset=["end","val"]).sort_values("end")
        if df.empty:
            continue
        last_date = df["end"].iloc[-1]
        if pd.notna(last_date) and last_date > best_last:
            best_last = last_date
            best_tag, best_df = tag, df.copy()
    return best_tag, best_df

# build standardized quarterly and annual revenue views, each with ['date','label','val']
def build_revenue_views(rev_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if rev_df is None or rev_df.empty:
        empty = pd.DataFrame(columns=["date","label","val"])
        return empty, empty

    # ensure helpful columns exist
    for c in ["form","fy","fp","frame","qtrs"]:
        if c not in rev_df.columns:
            rev_df[c] = pd.NA

    df = rev_df.copy()
    df["form"] = df["form"].astype(str).str.upper()
    df["fp"]   = df["fp"].astype(str).str.upper()
    df = df.sort_values("end")

    # quarterly view (convert ytd 10-q to true quarter if needed)
    q = df[(df["form"]=="10-Q") | (df["fp"].isin(["Q1","Q2","Q3","Q4"]))].copy()
    if q.empty:
        q = df.copy()
    q["is_ytd"] = q["frame"].astype(str).str.contains("YTD", na=False)
    with np.errstate(all='ignore'):
        q["is_ytd"] = q["is_ytd"] | (pd.to_numeric(q["qtrs"], errors="coerce").fillna(1) > 1)
    q["qnum"] = q["fp"].map({"Q1":1,"Q2":2,"Q3":3,"Q4":4})
    q["qnum"] = q["qnum"].fillna(pd.to_datetime(q["end"]).dt.quarter)
    q["year_key"] = q["fy"].fillna(pd.to_datetime(q["end"]).dt.year)
    q = q.sort_values(["year_key","qnum","end"])
    q["val_q"] = q["val"]
    ytd_mask = q["is_ytd"].fillna(False)
    if ytd_mask.any():
        q.loc[ytd_mask, "val_q"] = q[ytd_mask].groupby("year_key")["val"].diff().fillna(q[ytd_mask]["val"])
    q_out = pd.DataFrame({
        "date":  q["end"].values,
        "label": ("Q" + q["qnum"].astype(int).astype(str) + " " + q["year_key"].astype(int).astype(str)).values,
        "val":   q["val_q"].astype(float).values
    }).dropna(subset=["date","val"]).drop_duplicates(subset=["label"], keep="last").sort_values("date")

    # annual view (one value per fiscal year, prefer 10-k)
    a = df[(df["form"]=="10-K") | (df["fp"]=="FY")].copy()
    if a.empty:
        a = df.copy()
        a["year_key"] = pd.to_datetime(a["end"]).dt.year
        a = a.sort_values(["year_key","end"]).drop_duplicates(subset=["year_key"], keep="last")
    else:
        a["year_key"] = a["fy"].fillna(pd.to_datetime(a["end"]).dt.year)
        a = a.sort_values(["year_key","end"]).drop_duplicates(subset=["year_key"], keep="last")
    a_out = pd.DataFrame({
        "date":  a["end"].values,
        "label": ("FY " + a["year_key"].astype(int).astype(str)).values,
        "val":   a["val"].astype(float).values
    }).dropna(subset=["date","val"]).drop_duplicates(subset=["label"], keep="last").sort_values("date")

    return q_out.reset_index(drop=True), a_out.reset_index(drop=True)

# tag map for other key metrics
GAAP_TAGS = {
    "net_income":   ["NetIncomeLoss"],
    "assets":       ["Assets"],
    "liabilities":  ["Liabilities"],
    "equity":       ["StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest","StockholdersEquity"],
    "cash":         ["CashAndCashEquivalentsAtCarryingValue"],
    "ocf":          ["NetCashProvidedByUsedInOperatingActivities"],
    "curr_assets":  ["AssetsCurrent"],
    "curr_liab":    ["LiabilitiesCurrent"],
    "eps_diluted":  ["EarningsPerShareDiluted"]
}

# get the latest numeric value across related tags (preferring usd or usd/shares for eps)
def latest_value_from_tags(facts: dict, tag_list: List[str], prefer_eps=False) -> Optional[float]:
    usg = facts.get("facts", {}).get("us-gaap", {})
    best_val, best_last = None, pd.Timestamp("1900-01-01")
    for tag in tag_list:
        node = usg.get(tag)
        if not node:
            continue
        units = node.get("units", {})
        if prefer_eps:
            unit_keys = [k for k in units if "usd/shares" in k.lower()]
            if not unit_keys:
                unit_keys = [k for k in units if "usd" in k.lower()]
        else:
            unit_keys = [k for k in units if "usd" in k.lower()]
        for uk in unit_keys:
            df = pd.DataFrame(units.get(uk, []))
            if df.empty or "end" not in df or "val" not in df:
                continue
            df["end"] = pd.to_datetime(df["end"], errors="coerce")
            df["val"] = pd.to_numeric(df["val"], errors="coerce")
            df = df.dropna(subset=["end","val"]).sort_values("end")
            if df.empty:
                continue
            last_date = df["end"].iloc[-1]
            last_val  = float(df["val"].iloc[-1])
            if pd.notna(last_date) and last_date > best_last:
                best_last, best_val = last_date, last_val
    return best_val

# simple classroom rating based on yoy revenue, net margin, and leverage
def toy_rating(yoy_rev, net_margin, debt_to_equity) -> Tuple[str, str]:
    # buy if strong growth and margins with moderate leverage; sell if negative signals or high leverage; else hold
    expl = []
    if yoy_rev is not None: expl.append(f"YoY revenue {yoy_rev:.1%}")
    if net_margin is not None: expl.append(f"net margin {net_margin:.1%}")
    if debt_to_equity is not None: expl.append(f"D/E {debt_to_equity:.2f}")
    if (yoy_rev is not None and yoy_rev >= 0.10) and (net_margin is not None and net_margin >= 0.10) and (debt_to_equity is not None and debt_to_equity < 1.5):
        return "Buy",  " | ".join(expl)
    if (yoy_rev is not None and yoy_rev <= 0) or (net_margin is not None and net_margin <= 0) or (debt_to_equity is not None and debt_to_equity > 2.5):
        return "Sell", " | ".join(expl)
    return "Hold", " | ".join(expl)

# formatting helpers (money, percent, and safe division)
def fmt(v, money=False, pct=False):
    if v is None or (isinstance(v, float) and np.isnan(v)): return "NA"
    if pct: return f"{v:.1%}"
    if money:
        x = abs(v)
        if x >= 1e12: return f"${v/1e12:.2f}T"
        if x >= 1e9:  return f"${v/1e9:.2f}B"
        if x >= 1e6:  return f"${v/1e6:.2f}M"
        if x >= 1e3:  return f"${v/1e3:.2f}K"
        return f"${v:,.0f}"
    return f"{v:,.2f}"

def safe_div(a, b):
    # safe division with guards for missing and zero denominators
    try:
        if a is None or b is None or b == 0: return None
        return float(a)/float(b)
    except Exception:
        return None

# stooq symbol candidates (for us tickers) and price fetch
def stooq_candidates(ticker: str):
    t = ticker.strip().lower()
    cands = [t, f"{t}.us"]
    if "." in t:
        td = t.replace(".", "-")
        cands += [td, f"{td}.us"]
    if t in {"brk.b","brk-b"}:
        cands += ["brk-b","brk-b.us"]
    seen, out = set(), []
    for s in cands:
        if s not in seen:
            seen.add(s); out.append(s)
    return out

# fetch daily close prices from stooq and return ['date','close']
def fetch_stock_price_stooq(ticker: str,
                            hosts=("https://stooq.com", "https://stooq.pl"),
                            intervals=("d",),
                            max_retries=2,
                            pause=0.15,
                            timeout=25) -> pd.DataFrame:
    header = "Date,Open,High,Low,Close,Volume"
    for host in hosts:
        for sym in stooq_candidates(ticker):
            for itv in intervals:
                url = f"{host}/q/d/l/?s={sym}&i={itv}"
                for _ in range(max_retries):
                    try:
                        time.sleep(pause)
                        r = requests.get(url, timeout=timeout)
                        if r.ok and header in r.text:
                            df = pd.read_csv(io.StringIO(r.text))
                            if {"Date","Close"}.issubset(df.columns) and len(df) > 0:
                                out = df[["Date","Close"]].rename(columns={"Date":"date","Close":"close"})
                                out["date"] = pd.to_datetime(out["date"], errors="coerce")
                                out = out.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
                                return out
                    except Exception:
                        time.sleep(pause)
                        continue
    return pd.DataFrame(columns=["date","close"])

# plotting helpers: revenue bars and price line with date sliders
def bokeh_revenue_with_slider(ticker: str, df: pd.DataFrame, title_suffix: str):
    # plot revenue bars over time with a date range slider; y-axis in billions usd
    if df is None or df.empty:
        return
    df_plot = df.copy()
    df_plot["val_b"] = df_plot["val"] / 1e9
    src = ColumnDataSource(dict(
        date  = df_plot["date"],
        val   = df_plot["val_b"],
        label = df_plot["label"]
    ))
    p = figure(x_axis_type="datetime", width=900, height=320,
               title=f"{ticker.upper()} — {title_suffix} Revenue (Billions USD, latest and interactive)")
    approx_days = 25 if "Quarter" in title_suffix else 120
    p.vbar(x="date", top="val", width=approx_days*24*60*60*1000, source=src)
    p.add_tools(HoverTool(tooltips=[("Period","@label"),("Revenue (B USD)","@val{0,0.00}")]))
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Revenue (Billions USD)"
    p.yaxis.formatter.use_scientific = False
    start = df_plot["date"].min()
    end   = df_plot["date"].max()
    drs = DateRangeSlider(title="Visible date range", value=(start, end), start=start, end=end, width=900)
    drs.js_link("value", p.x_range, "start", attr_selector=0)
    drs.js_link("value", p.x_range, "end",   attr_selector=1)
    show(column(p, drs))

def bokeh_price_with_slider(ticker: str, price_df: pd.DataFrame):
    # plot daily close with a date range slider
    if price_df is None or price_df.empty:
        print(" Could not retrieve stock prices for this ticker.")
        return
    src = ColumnDataSource(dict(
        date  = price_df["date"],
        close = price_df["close"].astype(float)
    ))
    p = figure(x_axis_type="datetime", width=900, height=320,
               title=f"{ticker.upper()} — Daily Close (interactive)")
    p.line(x="date", y="close", source=src, line_width=2)
    p.add_tools(HoverTool(tooltips=[("Date","@date{%F}"),("Close","@close{0,0.00}")],
                          formatters={"@date":"datetime"}))
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Price"
    p.yaxis.formatter.use_scientific = False
    start = price_df["date"].min()
    end   = price_df["date"].max()
    drs = DateRangeSlider(title="Visible date range", value=(start, end), start=start, end=end, width=900)
    drs.js_link("value", p.x_range, "start", attr_selector=0)
    drs.js_link("value", p.x_range, "end",   attr_selector=1)
    show(column(p, drs))

# main driver: fetch facts, build series, print stats, plot charts
def analyze_company(ticker: str):
    # end-to-end: map ticker to cik, pull facts, build revenue views, print stats, draw charts, and give a toy label
    print(f"Analyzing {ticker.upper()} from SEC EDGAR...\n")
    tickers_df = get_ticker_map()
    cik = cik_for_ticker(ticker, tickers_df)
    if not cik:
        print("Ticker not found in SEC list.")
        return
    facts = get_companyfacts(cik)
    tag, rev_df = pick_freshest_revenue_tag(facts, REVENUE_TAGS)
    if rev_df is None or rev_df.empty:
        print("No revenue data found.")
        return
    q_view, a_view = build_revenue_views(rev_df)

    # latest snapshots
    latest_rev = rev_df.sort_values("end")["val"].iloc[-1] if not rev_df.empty else None
    ni_latest = latest_value_from_tags(facts, GAAP_TAGS["net_income"])
    assets = latest_value_from_tags(facts, GAAP_TAGS["assets"])
    liab = latest_value_from_tags(facts, GAAP_TAGS["liabilities"])
    if isinstance(liab, type(None)):
        liab = latest_value_from_tags(facts, GAAP_TAGS["liabilities"])
    equity = latest_value_from_tags(facts, GAAP_TAGS["equity"])
    cash = latest_value_from_tags(facts, GAAP_TAGS["cash"])
    ocf = latest_value_from_tags(facts, GAAP_TAGS["ocf"])
    ca = latest_value_from_tags(facts, GAAP_TAGS["curr_assets"])
    cl = latest_value_from_tags(facts, GAAP_TAGS["curr_liab"])
    epsd = latest_value_from_tags(facts, GAAP_TAGS["eps_diluted"], prefer_eps=True)

    # ratios
    current_ratio = safe_div(ca, cl)
    debt_to_equity = safe_div(liab, equity)
    net_margin = safe_div(ni_latest, latest_rev)

    # yoy revenue from annual series (if available)
    yoy_revenue = None
    if not a_view.empty and len(a_view) >= 2:
        a0, a1 = a_view["val"].iloc[-2], a_view["val"].iloc[-1]
        if a0 and a0 != 0:
            yoy_revenue = (a1 - a0) / a0

    # print key stats
    print("--- Key Stats ---")
    print(f"Latest Revenue: {fmt(latest_rev, money=True)}")
    print(f"Latest Net Income: {fmt(ni_latest,  money=True)}")
    print(f"Total Assets: {fmt(assets, money=True)}   |  Total Liabilities: {fmt(liab, money=True)}")
    print(f"Stockholders' Equity: {fmt(equity, money=True)}   |  Cash & Eq:          {fmt(cash, money=True)}")
    print(f"Operating Cash Flow: {fmt(ocf,   money=True)}")
    print(f"Current Ratio: {fmt(current_ratio)}")
    print(f"Debt to Equity: {fmt(debt_to_equity)}")
    print(f"Net Margin: {fmt(net_margin, pct=True)}")
    print(f"Diluted EPS: {fmt(epsd)}")
    print(f"YoY Revenue Growth: {fmt(yoy_revenue, pct=True)}")

    # classroom recommender
    def local_toy_rating(yoy_rev, net_margin, debt_to_equity):
        expl = []
        if yoy_rev is not None: expl.append(f"YoY revenue {yoy_rev:.1%}")
        if net_margin is not None: expl.append(f"net margin {net_margin:.1%}")
        if debt_to_equity is not None: expl.append(f"D/E {debt_to_equity:.2f}")
        if (yoy_rev is not None and yoy_rev >= 0.10) and (net_margin is not None and net_margin >= 0.10) and (debt_to_equity is not None and debt_to_equity < 1.5):
            return "Buy",  " | ".join(expl)
        if (yoy_rev is not None and yoy_rev <= 0) or (net_margin is not None and net_margin <= 0) or (debt_to_equity is not None and debt_to_equity > 2.5):
            return "Sell", " | ".join(expl)
        return "Hold", " | ".join(expl)

    label, reason = local_toy_rating(yoy_revenue, net_margin, debt_to_equity)
    print(f"\n--- Recommender (toy / classroom only) ---")
    print(f"Rating: {label}  |  {reason}\n")

    # plots
    if not q_view.empty:
        bokeh_revenue_with_slider(ticker, q_view, "Quarterly")
    else:
        print("No quarterly revenue series available.")
    if not a_view.empty:
        bokeh_revenue_with_slider(ticker, a_view, "Annual")
    else:
        print("No annual revenue series available.")
    px = fetch_stock_price_stooq(ticker)
    bokeh_price_with_slider(ticker, px)


In [20]:
ticker_input = "AAPL"   # ← change this to the ticker you want
analyze_company(ticker_input) # run the analysis function with the specified ticker

Analyzing AAPL from SEC EDGAR...

--- Key Stats ---
Latest Revenue: $94.04B
Latest Net Income: $23.43B
Total Assets: $331.50B   |  Total Liabilities: $265.67B
Stockholders' Equity: $65.83B   |  Cash & Eq:          $36.27B
Operating Cash Flow: $81.75B
Current Ratio: 0.87
Debt to Equity: 4.04
Net Margin: 24.9%
Diluted EPS: 1.57
YoY Revenue Growth: 2.0%

--- Recommender (toy / classroom only) ---
Rating: Sell  |  YoY revenue 2.0% | net margin 24.9% | D/E 4.04



---


##Regression

The regression module takes the standardized revenue series from EDGAR and turns it into a simple, transparent forecast. After you choose a frequency, quarterly or annual, the code orders the revenue points by date and fits a straight line with ordinary least squares where time is the only input. In other words, it models revenue as y = a + b·t, with t running from the first observed period to the last. It computes the fitted values for the observed points, the residual variance, and then builds a 95 percent confidence band around the fitted line to show uncertainty in the average relationship. It also issues a one-step-ahead prediction for the next quarter or next fiscal year and pairs that forecast with a 95 percent prediction interval, which is wider than the confidence band because it accounts for both model error and noise in new observations. The chart plots the actual revenue points, the best fit line, the confidence band, and a single forecast marker for the next period with its prediction range. The module requires at least four observations to avoid unstable fits and it falls back to the other frequency if the chosen one is too short. This approach keeps the trend estimate easy to interpret, makes uncertainty visible, and complements the descriptive metrics by suggesting a plausible next revenue value rather than only describing the past.

In [21]:
# User settings
TICKER = "AAPL"          # <<< change this - this is the stock ticker symbol to analyze
FREQUENCY = "Quarterly"  # "Quarterly" or "Annual" (code will fall back if not enough data) - this sets the frequency for the revenue analysis
SEC_USER_AGENT = os.getenv("SEC_USER_AGENT", "Ahmad Alshikh Menou aa11184@nyu.edu (academic use)") # this is required by the SEC for API requests
PAUSE_S = 0.2 # this is a brief pause to avoid overloading the SEC servers with requests
HEADERS = {"User-Agent": SEC_USER_AGENT, "Accept-Encoding": "gzip, deflate"} # these are the headers to be included in HTTP requests

In [22]:
# revenue tags to try; we pick the one with the most recent period so charts are current
REVENUE_TAGS = ["RevenueFromContractWithCustomerExcludingAssessedTax", "SalesRevenueNet", "Revenues"]

# small http helper that politely fetches json from a url (sec requires a user-agent)
def get_json(url: str) -> dict:
    time.sleep(PAUSE_S)
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

# build the official sec ticker → cik map, with a fallback if the primary file is unavailable
def get_ticker_map() -> pd.DataFrame:
    primary = "https://www.sec.gov/files/company_tickers.json"
    try:
        data = get_json(primary)
        df = pd.DataFrame(list(data.values()))
        df["cik"] = df["cik_str"].astype(str).str.zfill(10)
        df = df.rename(columns={"ticker":"ticker", "title":"company"})
        return df[["ticker","cik","company"]]
    except Exception:
        fallback = "https://www.sec.gov/files/company_tickers_exchange.json"
        data = get_json(fallback)
        df = pd.DataFrame(data)
        df["cik"] = df["cik"].astype(str).str.zfill(10)
        df["ticker"] = df["ticker"].str.upper()
        df = df.rename(columns={"title":"company"})
        return df[["ticker","cik","company"]]

# look up the cik for a given ticker (case-insensitive)
def cik_for_ticker(ticker: str, tickers_df: pd.DataFrame) -> Optional[str]:
    row = tickers_df[tickers_df["ticker"].str.upper() == ticker.upper()]
    return None if row.empty else row["cik"].iloc[0]

# download the company xbrl "facts" json from sec edgar
def get_companyfacts(cik: str) -> dict:
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    return get_json(url)

# choose the freshest revenue tag and return a tidy dataframe of its observations
def pick_freshest_revenue_tag(facts: dict, tags: List[str]) -> Tuple[Optional[str], Optional[pd.DataFrame]]:
    usg = facts.get("facts", {}).get("us-gaap", {})
    best_tag, best_df = None, None
    best_last = pd.Timestamp("1900-01-01")
    for tag in tags:
        node = usg.get(tag)
        if not node:
            continue
        units = node.get("units", {})
        unit_keys = [k for k in units.keys() if "usd" in k.lower()]
        if not unit_keys:
            continue
        rows = []
        for uk in unit_keys:
            rows.extend(units.get(uk, []))
        df = pd.DataFrame(rows)
        if df.empty or "end" not in df or "val" not in df:
            continue
        df["end"] = pd.to_datetime(df["end"], errors="coerce")
        df["val"] = pd.to_numeric(df["val"], errors="coerce")
        df = df.dropna(subset=["end","val"]).sort_values("end")
        if df.empty:
            continue
        last_date = df["end"].iloc[-1]
        if pd.notna(last_date) and last_date > best_last:
            best_last, best_tag, best_df = last_date, tag, df.copy()
    return best_tag, best_df

# build standardized quarterly and annual revenue views
# quarterly converts ytd 10-q values to true per-quarter by differencing within fiscal year
# annual keeps a single value per fiscal year (prefers 10-k if available)
def build_revenue_views(rev_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if rev_df is None or rev_df.empty:
        return (pd.DataFrame(columns=["label","val","end"]),
                pd.DataFrame(columns=["label","val","end"]))

    for c in ["form","fy","fp","frame","qtrs"]:
        if c not in rev_df.columns:
            rev_df[c] = pd.NA

    df = rev_df.copy()
    df["form"] = df["form"].astype(str).str.upper()
    df["fp"]   = df["fp"].astype(str).str.upper()
    df = df.sort_values("end")

    # quarterly view
    q = df[(df["form"]=="10-Q") | (df["fp"].isin(["Q1","Q2","Q3","Q4"]))].copy()
    if q.empty:
        q = df.copy()
    q["is_ytd"] = q["frame"].astype(str).str.contains("YTD", na=False)
    with np.errstate(all='ignore'):
        q["is_ytd"] = q["is_ytd"] | (pd.to_numeric(q["qtrs"], errors="coerce").fillna(1) > 1)
    q["qnum"] = q["fp"].map({"Q1":1,"Q2":2,"Q3":3,"Q4":4})
    q["qnum"] = q["qnum"].fillna(pd.to_datetime(q["end"]).dt.quarter)
    q["year_key"] = q["fy"].fillna(pd.to_datetime(q["end"]).dt.year)
    q = q.sort_values(["year_key","qnum","end"])
    q["val_q"] = q["val"]
    ytd_mask = q["is_ytd"].fillna(False)
    if ytd_mask.any():
        q.loc[ytd_mask, "val_q"] = q[ytd_mask].groupby("year_key")["val"].diff().fillna(q[ytd_mask]["val"])
    q["label"] = "Q" + q["qnum"].astype(int).astype(str) + " " + q["year_key"].astype(int).astype(str)
    q_view = q[["label","val_q","end"]].rename(columns={"val_q":"val"})
    q_view = q_view.dropna(subset=["val"]).drop_duplicates(subset=["label"], keep="last")
    q_view = q_view.sort_values("end").reset_index(drop=True)

    # annual view
    a = df[(df["form"]=="10-K") | (df["fp"]=="FY")].copy()
    if a.empty:
        a = df.copy()
        a["year_key"] = pd.to_datetime(a["end"]).dt.year
        a = a.sort_values(["year_key","end"]).drop_duplicates(subset=["year_key"], keep="last")
    else:
        a["year_key"] = a["fy"].fillna(pd.to_datetime(a["end"]).dt.year)
        a = a.sort_values(["year_key","end"]).drop_duplicates(subset=["year_key"], keep="last")
    a["label"] = "FY " + a["year_key"].astype(int).astype(str)
    a_view = a[["label","val","end"]].dropna(subset=["val"]).drop_duplicates(subset=["label"], keep="last")
    a_view = a_view.sort_values("end").reset_index(drop=True)

    return q_view, a_view

# ordinary least squares fit with 95% confidence band for a simple time index
def _ols_fit_with_intervals(y: np.ndarray):
    n = len(y)
    x = np.arange(n, dtype=float)
    xbar = x.mean(); ybar = y.mean()
    Sxx = ((x - xbar)**2).sum()
    if n < 4 or Sxx == 0:
        return None
    b = ((x - xbar) * (y - ybar)).sum() / Sxx
    a = ybar - b * xbar
    yhat = a + b * x
    resid = y - yhat
    RSS = (resid**2).sum()
    s2 = RSS / (n - 2)
    tcrit = 1.96
    se_mean = np.sqrt(s2 * (1.0/n + ((x - xbar)**2) / Sxx))
    ci_low  = yhat - tcrit * se_mean
    ci_high = yhat + tcrit * se_mean
    return {"a":a, "b":b, "yhat":yhat, "ci_low":ci_low, "ci_high":ci_high,
            "s2":s2, "xbar":xbar, "Sxx":Sxx, "tcrit":tcrit, "n":n}

# one-step-ahead prediction from the ols fit with a 95% prediction interval
def _ols_predict_next(fit: dict):
    n = fit["n"]; x0 = float(n)
    a, b = fit["a"], fit["b"]
    s2, xbar, Sxx, tcrit = fit["s2"], fit["xbar"], fit["Sxx"], fit["tcrit"]
    y0 = a + b * x0
    se_pred = np.sqrt(s2 * (1.0 + 1.0/n + ((x0 - xbar)**2) / Sxx))
    return y0, y0 - tcrit*se_pred, y0 + tcrit*se_pred

# plot the regression line, confidence band, and a next-step forecast for the chosen revenue series
def plot_revenue_regression(ticker: str, series_df: pd.DataFrame, freq_label: str):
    if series_df is None or series_df.empty or len(series_df) < 4:
        print("Not enough periods to fit a regression (need at least 4).")
        return

    s = series_df.sort_values("end").copy()
    labels = s["label"].tolist()
    y = (s["val"].astype(float) / 1e9).values

    fit = _ols_fit_with_intervals(y)
    if fit is None:
        print("Regression could not be fit.")
        return

    if freq_label.lower().startswith("q"):
        last = labels[-1]
        m = re.match(r"Q(\d)\s+(\d{4})", last)
        if m:
            qn = int(m.group(1)); yr = int(m.group(2))
            qn_next = 1 if qn == 4 else qn + 1
            yr_next = yr + 1 if qn == 4 else yr
            next_label = f"Q{qn_next} {yr_next}"
        else:
            next_label = "Next"
    else:
        last = labels[-1]
        m = re.match(r"FY\s+(\d{4})", last)
        next_label = f"FY {int(m.group(1))+1}" if m else "Next"

    y_next, y_next_lo, y_next_hi = _ols_predict_next(fit)

    src_hist = ColumnDataSource(data=dict(
        label=labels,
        y=y,
        yhat=fit["yhat"],
        ci_l=fit["ci_low"],
        ci_h=fit["ci_high"]
    ))
    x_all = labels + [next_label]
    src_next = ColumnDataSource(data=dict(
        label=[next_label],
        yhat=[float(y_next)],
        lo=[float(y_next_lo)],
        hi=[float(y_next_hi)]
    ))

    p = figure(width=950, height=360, x_range=x_all,
               title=f"{ticker.upper()} — {freq_label} Revenue Regression (Billions USD)")
    p.varea(x="label", y1="ci_l", y2="ci_h", source=src_hist, fill_alpha=0.2)
    p.line(x="label", y="yhat", source=src_hist, line_width=2, legend_label="best-fit (mean)")
    p.circle(x="label", y="y", source=src_hist, size=7, legend_label="actual")
    p.circle(x="label", y="yhat", source=src_next, size=9, color="black", legend_label="next forecast")
    p.segment(x0="label", y0="lo", x1="label", y1="hi", source=src_next, line_width=3, color="black")

    p.add_tools(HoverTool(tooltips=[
        ("period","@label"),
        ("actual (b usd)","@y{0,0.00}"),
        ("fit (b usd)","@yhat{0,0.00}")
    ]))
    p.xaxis.axis_label = "period"
    p.yaxis.axis_label = "revenue (billions usd)"
    p.yaxis.formatter.use_scientific = False
    p.legend.location = "top_left"
    show(p)

# driver that fetches revenue, prepares the view, runs regression, and plots it
def run_revenue_regression(ticker: str, freq_choice="Quarterly"):
    print(f"Revenue regression for {ticker.upper()}  |  View: {freq_choice}")
    tickers_df = get_ticker_map()
    cik = cik_for_ticker(ticker, tickers_df)
    if not cik:
        print("Ticker not found in SEC map.")
        return
    facts = get_companyfacts(cik)
    tag, rev_df = pick_freshest_revenue_tag(facts, REVENUE_TAGS)
    if rev_df is None or rev_df.empty:
        print("No revenue data found.")
        return
    q_view, a_view = build_revenue_views(rev_df)

    # choose quarterly or annual series, falling back to whichever has enough observations
    series, label = None, None
    if freq_choice.lower().startswith("q") and len(q_view) >= 4:
        series, label = q_view, "Quarterly"
    elif freq_choice.lower().startswith("a") and len(a_view) >= 4:
        series, label = a_view, "Annual"
    else:
        if len(q_view) >= len(a_view) and len(q_view) >= 4:
            series, label = q_view, "Quarterly"
        elif len(a_view) >= 4:
            series, label = a_view, "Annual"
        else:
            print("Not enough periods in either series to fit a regression (need at least 4).")
            return

    # small preview table (in billions)
    print("\nLast few periods used (Billions USD):")
    series_b = series.copy()
    series_b["val"] = series_b["val"] / 1e9
    series_b = series_b.rename(columns={"val": "Revenue (B USD)"})
    display(series_b.tail(8))

    plot_revenue_regression(ticker, series, label)

# run the regression (expects TICKER and FREQUENCY to be defined elsewhere)
run_revenue_regression(TICKER, freq_choice=FREQUENCY)


Revenue regression for AAPL  |  View: Quarterly

Last few periods used (Billions USD):


,label,Revenue (B USD),end
13,Q2 2023,94.836,2023-04-01
14,Q3 2023,81.797,2023-07-01
15,Q1 2024,119.575,2023-12-30
16,Q2 2024,90.753,2024-03-30
17,Q3 2024,85.777,2024-06-29
18,Q1 2025,124.300,2024-12-28
19,Q2 2025,95.359,2025-03-29
20,Q3 2025,94.036,2025-06-28


# Portfolio
This is an attempt of a portfolio builder that takes into account the user's risk tolerance input and in return (from a limited universe of US equities).
This code chunk yields a diversified stock portfolio for you and shows how it would have performed. It pulls recent daily prices for a broad set of well known US companies, lines the data up on the same calendar, and keeps the history current to the most recent trading day.

Your positions are sized with a simple blend of three ideas: equal weight, inverse volatility, and a risk-return tilt from past data. The risk_aversion_input you choose controls that blend. Lower values lean toward growth. Higher values lean toward stability. A cap keeps any single stock from getting too large.

You get three things to read at a glance: a bar chart of your weights, a scatter of risk versus return with bubble size equal to weight, and a growth of one dollar chart with a date slider. Below the plots you will see summary stats like CAGR, volatility, Sharpe, and max drawdown. Change risk_aversion_input, start_date, or end_date to explore different setups and see how the mix and results respond.

In [23]:
# user controls (risk appetite and other variables)
risk_aversion_input = 0.5          # numeric 0.5..10 or one of: "very low","low","medium","high","very high"
start_date = "2013-01-01"          # first day in the backtest
end_date   = None                  # last day (None = most recent available)
risk_free  = 0.02                  # annual risk-free rate used in sharpe ratio

In [24]:
output_notebook()  # bokeh plots render inline in the notebook

# simple runtime knobs we actually use in this file
universe_cap   = 48
max_workers    = 8
http_timeout   = 20
http_pause_s   = 0.05
min_rows_each  = 280

# covariance / weights controls (stability + diversification)
cov_shrink       = 0.10
cap_name_weight  = None            # None => dynamic cap = max(4%, 1.5/N)

# data alignment hygiene (reach most recent trading day with minimal gaps)
quorum_coverage   = 0.85           # keep dates where ≥ 85% of names have data
stale_cutoff_days = 20             # drop tickers lagging the freshest name by > 20 days
ffill_limit       = 1              # forward-fill up to 1 day for tiny gaps

# reproducibility
random.seed(42)
np.random.seed(42)

# starting universe (diversified across sectors)
full_universe = [
    # tech / comm
    "AAPL","MSFT","GOOGL","AMZN","NVDA","META","TSLA","CRM","ORCL","IBM","CSCO","QCOM","TXN","ADI","AMD","AVGO",
    "NFLX","DIS","CMCSA",
    # financials
    "BRK.B","JPM","BAC","WFC","C","GS","MS","V","MA","AXP","PYPL",
    # health care
    "JNJ","PFE","MRK","UNH","LLY","ABT","TMO","CVS","HUM",
    # energy & utilities
    "XOM","CVX","COP","SLB","OXY","DVN","NEE","DUK","SO","AEP",
    # staples / discretionary / industrials
    "PG","KO","PEP","WMT","COST","HD","LOW","MCD","SBUX","NKE","BA","CAT","DE","GE","UPS","FDX"
]

# pick a round-robin subset up to the requested cap
def diversified_subset(universe, cap):
    blocks = [
        universe[0:18],          # tech/comm
        universe[18:29],         # financials
        universe[29:38],         # health care
        universe[38:48],         # energy/utilities
        universe[48:]            # staples/discr/industrials
    ]
    if cap is None or cap >= len(universe):
        return list(dict.fromkeys(universe))
    sel = []; i = 0
    while len(sel) < cap:
        for b in blocks:
            if i < len(b):
                sel.append(b[i])
                if len(sel) >= cap: break
        i += 1
        if i > max(len(b) for b in blocks): break
    return list(dict.fromkeys(sel))

user_universe = diversified_subset(full_universe, universe_cap)

# price downloads from stooq (no local cache)
def _stooq_try_symbols(ticker: str) -> list:
    t = ticker.lower()
    if t in {"brk.b", "brk-b"}:
        return ["brk-b.us", "brk-b"]
    return [t, f"{t}.us"]

##download one ticker’s daily closes as ['date', TICKER]
def fetch_stooq_one(ticker: str) -> pd.DataFrame:
    tick = ticker.upper()
    for sym in _stooq_try_symbols(tick):
        url = f"https://stooq.com/q/d/l/?s={sym}&i=d"
        try:
            time.sleep(http_pause_s + random.random()*http_pause_s)
            r = requests.get(url, timeout=http_timeout)
            if r.ok and "Date,Open,High,Low,Close,Volume" in r.text:
                df = pd.read_csv(io.StringIO(r.text))
                if {"Date","Close"}.issubset(df.columns) and len(df) > 0:
                    out = df[["Date","Close"]].rename(columns={"Date":"date","Close":tick})
                    out["date"] = pd.to_datetime(out["date"], errors="coerce")
                    out = out.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
                    return out
        except Exception:
            continue
    return pd.DataFrame(columns=["date", tick])

##download one ticker’s daily closes as ['date', TICKER]
def fetch_many_prices(tickers: list) -> dict:
    out = {}
    uniq = list(dict.fromkeys(map(str.upper, tickers)))
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(fetch_stooq_one, t): t for t in uniq}
        for fut in as_completed(futs):
            t = futs[fut]
            try:
                out[t] = fut.result()
            except Exception:
                out[t] = pd.DataFrame(columns=["date", t])
    return out

# date trimming to reach latest trading day
def _trim_dates(df: pd.DataFrame, start: str, end: str) -> pd.DataFrame:
    if start:
        df = df[df["date"] >= pd.Timestamp(start)]
    if end:
        df = df[df["date"] <= pd.Timestamp(end)]
    return df

def build_price_matrix(tickers: list,
                       start=None,
                       end=None,
                       min_rows_each=min_rows_each) -> pd.DataFrame:
# we folowed these steps: fetch → prefilter → drop stale → outer-join → small ffill → quorum mask
    frames = fetch_many_prices(tickers)

    clean = {}
    last_dates = {}
    for t, df in frames.items():
        if df.empty:
            continue
        df = _trim_dates(df, start, end)
        if len(df) < min_rows_each:
            continue
        s = df[t].dropna()
        if len(s) < 2 or s.nunique() <= 1:
            continue
        dfi = df.set_index("date")[[t]].sort_index()
        clean[t] = dfi
        last_dates[t] = dfi.index.max()

    if not clean:
        print("no usable price series after prefilter.")
        return pd.DataFrame()

    max_last = max(last_dates.values())
    fresh = {t: d for t, d in clean.items() if (max_last - last_dates[t]).days <= stale_cutoff_days}
    if len(fresh) < 5:
        fresh = clean

    px = pd.concat(list(fresh.values()), axis=1, join="outer").sort_index()
    px = px.ffill(limit=ffill_limit)

    coverage = px.notna().sum(axis=1) / px.shape[1]
    px = px[coverage >= quorum_coverage]
    return px

# risk aversion mapping
def parse_risk_aversion(user_input) -> float:
    if isinstance(user_input, (int, float)):
        return float(np.clip(user_input, 0.5, 10.0))
    s = str(user_input).strip().lower()
    mapping = {
        "very low":0.5, "vl":0.5,
        "low":1.0, "l":1.0,
        "medium":3.0, "med":3.0, "m":3.0,
        "high":6.0, "h":6.0,
        "very high":10.0, "vh":10.0
    }
    return mapping.get(s, 3.0)

# diversified weights (equal-weight + inverse-vol + mean-variance blend with caps)
# we used some of our previous finance knowledge from other classes to map out the right variables relevant to the portfolio
def diversified_weights(px: pd.DataFrame, A: float, shrink=cov_shrink, cap=cap_name_weight, passes=4) -> pd.Series:
    tickers = list(px.columns)
    n = len(tickers)
    rets = px.pct_change().dropna(how="all")
    if rets.empty or n == 0:
        return pd.Series([], dtype="float64")

    mu  = rets.mean(skipna=True) * 252.0
    cov = rets.cov()            * 252.0

    diag_cov = np.diag(np.diag(cov.values))
    Sigma = (1 - shrink) * cov.values + shrink * diag_cov
    try:
        invS = np.linalg.inv(Sigma)
    except np.linalg.LinAlgError:
        invS = np.linalg.pinv(Sigma)

    mv_raw = invS @ mu.values
    mv = pd.Series(mv_raw, index=tickers).clip(lower=0)
    mv = mv / (mv.sum() if mv.sum() > 0 else 1)

    vol = rets.std(skipna=True) * np.sqrt(252.0)
    iv = (1.0 / vol.replace(0, np.nan)).fillna(0.0)
    iv = iv / (iv.sum() if iv.sum() > 0 else 1)

    eq = pd.Series(1.0 / n, index=tickers)

    t = float(np.clip((A - 0.5) / (10.0 - 0.5), 0.0, 1.0))
    gamma_mv = 0.55 * (1 - t) + 0.10
    beta_iv  = 0.55 * t       + 0.20
    alpha_eq = max(0.0, 1.0 - (gamma_mv + beta_iv))
    mv_adj = mv / max(A, 1e-6)

    w = alpha_eq * eq + beta_iv * iv + gamma_mv * mv_adj
    w = w.clip(lower=0)
    w = w / (w.sum() if w.sum() > 0 else 1)

    if cap is None:
        cap = max(0.04, 1.5 / n)

    for _ in range(passes):
        over = w[w > cap]
        if over.empty:
            break
        excess = (over - cap).sum()
        w[over.index] = cap
        under = w[w < cap]
        if under.empty or excess <= 0:
            break
        share = under / under.sum()
        w[under.index] = (under + excess * share).values
        w = w / w.sum()
    return w

# portfolio returns with daily re-normalization over available names
def _portfolio_returns_renormalized(px: pd.DataFrame, w: pd.Series) -> pd.Series:
    rets = px.pct_change()
    w = w.reindex(px.columns).fillna(0.0)
    pr_vals, idxs = [], []
    for idx, row in rets.iterrows():
        avail = row.dropna()
        if avail.empty:
            continue
        w_sub = w.reindex(avail.index).fillna(0.0)
        tot = w_sub.sum()
        if tot <= 0:
            continue
        pr_vals.append(float((avail * (w_sub / tot)).sum()))
        idxs.append(idx)
    if not pr_vals:
        return pd.Series(dtype="float64")
    return pd.Series(pr_vals, index=pd.Index(idxs, name=rets.index.name))

# summary metrics (growth, cagr, vol, sharpe, max drawdown)
def portfolio_metrics(px: pd.DataFrame, weights: pd.Series, rf_annual=risk_free):
    rets = px.pct_change()
    if not rets.empty:
        mu = rets.mean(skipna=True) * 252
        sigma = rets.std(skipna=True) * np.sqrt(252)
    else:
        mu = pd.Series(np.nan, index=px.columns, dtype="float64")
        sigma = pd.Series(np.nan, index=px.columns, dtype="float64")

    pr = _portfolio_returns_renormalized(px, weights)
    if pr.empty:
        return {"growth": pd.Series(dtype="float64"), "daily_returns": pr,
                "CAGR": np.nan, "Volatility": np.nan, "Sharpe": np.nan, "MaxDD": np.nan,
                "mu": mu, "sigma": sigma}

    growth = (1 + pr).cumprod()
    n = len(pr)
    cagr = growth.iloc[-1] ** (252.0 / n) - 1.0 if n > 0 else np.nan
    vol  = pr.std() * np.sqrt(252) if n > 1 else np.nan
    rf_d = (1 + rf_annual) ** (1/252) - 1
    sharpe = (pr.mean() - rf_d) / pr.std() * np.sqrt(252) if pr.std() > 0 else np.nan
    roll_max = growth.cummax()
    dd = growth / roll_max - 1.0
    maxdd = dd.min() if len(dd) else np.nan

    return {"growth": growth, "daily_returns": pr,
            "CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": maxdd,
            "mu": mu, "sigma": sigma}

# charts: weights bar, growth with date slider, risk/return scatter
def bokeh_weights_bar(weights: pd.Series, mu: pd.Series=None, sigma: pd.Series=None, title="Suggested Weights"):
    d = pd.DataFrame({"ticker": weights.index, "weight": weights.values})
    if isinstance(mu, pd.Series):    d["exp_return"] = mu.reindex(weights.index).values
    if isinstance(sigma, pd.Series): d["vol"]        = sigma.reindex(weights.index).values
    d = d.sort_values("weight", ascending=False).reset_index(drop=True)
    src = ColumnDataSource(d)
    p = figure(x_range=d["ticker"].tolist(), width=900, height=330, title=title, toolbar_location="above")
    p.vbar(x="ticker", top="weight", width=0.8, source=src)
    tips = [("Ticker","@ticker"), ("Weight","@weight{0.00%}")]
    if "exp_return" in d: tips.append(("Exp. Return","@exp_return{0.00%}"))
    if "vol" in d:        tips.append(("Volatility","@vol{0.00%}"))
    p.add_tools(HoverTool(tooltips=tips))
    p.yaxis.axis_label = "Portfolio Weight"
    p.xaxis.major_label_orientation = 0.9
    show(p)

## line chart + date range slider that controls the visible window
def bokeh_growth_line_with_slider(growth: pd.Series, title="Portfolio Growth ($1)"):

    if not isinstance(growth, pd.Series) or growth.empty:
        print("skipping growth chart: not enough data.")
        return
    g = growth.sort_index()
    src = ColumnDataSource(dict(date=g.index, val=g.values))

    p = figure(x_axis_type="datetime", width=900, height=330, title=title, toolbar_location="above")
    p.line(x="date", y="val", source=src, line_width=2)
    p.add_tools(HoverTool(tooltips=[("Date","@date{%F}"), ("Value","@val{0.000}")],
                          formatters={"@date":"datetime"}))
    p.xaxis.axis_label = "Date"; p.yaxis.axis_label = "Value of $1"

    start = g.index.min()
    end   = g.index.max()
    drs = DateRangeSlider(title="visible date range", value=(start, end), start=start, end=end, width=900)
    drs.js_link("value", p.x_range, "start", attr_selector=0)
    drs.js_link("value", p.x_range, "end",   attr_selector=1)

    show(column(p, drs))

def bokeh_risk_return_scatter(mu: pd.Series, sigma: pd.Series, weights: pd.Series, title="Constituent Risk/Return"):
    if not (isinstance(mu, pd.Series) and isinstance(sigma, pd.Series)):
        print("skipping risk/return scatter: mu/sigma unavailable.")
        return
    d = pd.DataFrame({
        "ticker": mu.index,
        "mu": mu.values,
        "sigma": sigma.reindex(mu.index).values,
        "w": weights.reindex(mu.index).fillna(0.0).values
    }).dropna()
    if d.empty:
        print("skipping risk/return scatter: not enough valid mu/sigma.")
        return
    d["size"] = 8 + 200 * d["w"]
    src = ColumnDataSource(d)
    p = figure(width=900, height=330, title=title, toolbar_location="above")
    p.circle(x="sigma", y="mu", size="size", source=src)
    p.add_tools(HoverTool(tooltips=[("Ticker","@ticker"),
                                    ("Exp. Return","@mu{0.00%}"),
                                    ("Vol","@sigma{0.00%}"),
                                    ("Weight","@w{0.00%}")]))
    p.xaxis.axis_label = "Volatility (annualized)"; p.yaxis.axis_label = "Expected Return (annualized)"
    show(p)

# driver: build portfolio and plot
def build_portfolio_interactive(user_tickers: list,
                                risk_aversion=risk_aversion_input,
                                start=start_date,
                                end=end_date,
                                rf=risk_free,
                                shrink=cov_shrink,
                                cap=cap_name_weight):
    A = parse_risk_aversion(risk_aversion)
    print(f"risk aversion A = {A}  (lower = more return-seeking, higher = more risk-averse)")
    print(f"requested universe size: {len(user_tickers)}")

    px = build_price_matrix(user_tickers, start=start, end=end, min_rows_each=min_rows_each)
    if px.empty or px.shape[1] < 5:
        print("need at least 5 tickers with sufficient overlapping history.")
        return None

    w = diversified_weights(px, A=A, shrink=shrink, cap=cap)
    if w.empty or (w.sum() <= 0):
        print("could not compute weights; try a later start date or smaller universe.")
        return None

    stats = portfolio_metrics(px, w, rf_annual=rf)

    print(f"\n portfolio built | names={len(w)}  rows={len(px)}")
    print("\ntop 20 weights:")
    for t, val in w.sort_values(ascending=False).head(20).items():
        print(f"  {t}: {val:.2%}")
    print("\nkey metrics:")
    print(f"  CAGR:        {stats['CAGR']:.2%}" if pd.notna(stats['CAGR']) else "  CAGR:        NA")
    print(f"  Volatility:  {stats['Volatility']:.2%}" if pd.notna(stats['Volatility']) else "  Volatility:  NA")
    print(f"  Sharpe:      {stats['Sharpe']:.2f}" if pd.notna(stats['Sharpe']) else "  Sharpe:      NA")
    print(f"  Max Drawdown:{stats['MaxDD']:.2%}" if pd.notna(stats['MaxDD']) else "  Max Drawdown: NA")

    bokeh_weights_bar(w, stats.get("mu"), stats.get("sigma"),
                      title=f"Weights — Diversified (A={A})")
    bokeh_risk_return_scatter(stats.get("mu"), stats.get("sigma"), w,
                              title="Constituent Risk/Return (size = weight)")
    bokeh_growth_line_with_slider(stats.get("growth"),
                                  title="Portfolio Growth ($1)")

# run once
result = build_portfolio_interactive(user_universe,
                                     risk_aversion=risk_aversion_input,
                                     start=start_date,
                                     end=end_date,
                                     rf=risk_free,
                                     shrink=cov_shrink,
                                     cap=cap_name_weight)


risk aversion A = 0.5  (lower = more return-seeking, higher = more risk-averse)
requested universe size: 48


/tmp/ipython-input-3469748774.py:166: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = px.pct_change().dropna(how="all")
/tmp/ipython-input-3469748774.py:238: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = px.pct_change()
/tmp/ipython-input-3469748774.py:219: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = px.pct_change()



 portfolio built | names=22  rows=3307

top 20 weights:
  BRK.B: 6.82%
  JNJ: 6.82%
  BAC: 6.82%
  MA: 6.82%
  NEE: 6.82%
  JPM: 6.82%
  TMO: 6.82%
  NVDA: 6.82%
  COST: 6.82%
  SO: 6.82%
  PEP: 5.11%
  XOM: 3.61%
  GOOGL: 3.45%
  COP: 3.34%
  AMZN: 3.25%
  MRK: 2.53%
  KO: 2.24%
  PFE: 1.89%
  IBM: 1.84%
  CMCSA: 1.80%

key metrics:
  CAGR:        19.40%
  Volatility:  15.50%
  Sharpe:      1.09
  Max Drawdown:-31.94%
